In [3]:
from azureml.core import Workspace
from azureml.core.authentication import MsiAuthentication
msi_auth = MsiAuthentication()

ws = Workspace(subscription_id="7704663e-8944-4300-b951-6462d41ab638", # put in .json
resource_group="arg-rt-pacedev-SpaceTech", # also in .json
workspace_name="aml-WS-spacetech001", # also
auth=msi_auth
)

keyvault = ws.get_default_keyvault()
conn_str = keyvault.get_secret("asartpacedevstvdsm001-connstr")

In [14]:
from azure.storage.blob import BlobServiceClient
blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")
blob_names = container_client.list_blobs(name_starts_with='training/traininginput_pv_v2')
for blob_name in blob_names:
    print(blob_name['name'])

training/traininginput_pv_v2/BM; TP; YA_251_251.csv
training/traininginput_pv_v2/BM_252_256.csv
training/traininginput_pv_v2/BM_257_261.csv
training/traininginput_pv_v2/BM_262_266.csv
training/traininginput_pv_v2/BM_267_271.csv
training/traininginput_pv_v2/BM_272_275.csv
training/traininginput_pv_v2/BM_276_278.csv
training/traininginput_pv_v2/BM_279_283.csv
training/traininginput_pv_v2/BM_284_287.csv
training/traininginput_pv_v2/BM_288_289.csv
training/traininginput_pv_v2/BM_290_291.csv
training/traininginput_pv_v2/BM_291_293.csv
training/traininginput_pv_v2/BM_293_295.csv
training/traininginput_pv_v2/BM_296_300.csv
training/traininginput_pv_v2/BM_301_305.csv
training/traininginput_pv_v2/BM_306_310.csv
training/traininginput_pv_v2/BM_311_315.csv
training/traininginput_pv_v2/BM_316_320.csv
training/traininginput_pv_v2/BM_321_325.csv
training/traininginput_pv_v2/BM_326_328.csv
training/traininginput_pv_v2/DD_-3_0.csv
training/traininginput_pv_v2/DD_103_107.csv
training/traininginput_pv_v

In [42]:
datetime.date(2021, 11, 1) >= datetime.date(2021, 12, 1)

False

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import traceback

veg_type = 'pv'

training_path = 'training/traininginput_{}_v2'.format(veg_type)

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")

end_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1)
start_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1) + relativedelta(months=-12)

months = []
month_dt = start_month_dt
while month_dt < end_month_dt:
    month_dt = month_dt + relativedelta(months=1)
    months.append(str(month_dt)[:7])

baseline_df = pd.DataFrame()


blob_names = container_client.list_blobs(name_starts_with=f'training/traininginput_{veg_type}_v2')

for blob_name in blob_names:
    print(blob_name['name'])
    blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
    df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
    target_df = target_df.append(df)

    

In [44]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO
import traceback

end_month = '2021-12'
veg_type = 'pv'

training_path = 'training/traininginput_{}_v2'.format(veg_type)

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")

end_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1)
start_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1) + relativedelta(months=-12)

months = []
month_dt = start_month_dt
while month_dt < end_month_dt:
    month_dt = month_dt + relativedelta(months=1)
    months.append(str(month_dt)[:7])

target_df = pd.DataFrame()

for month in months:

    blob_names = container_client.list_blobs(name_starts_with=f'{month[:4]}/{month[5:]}/fused/{veg_type}_v1')

    for blob_name in blob_names:
        print(blob_name['name'])
        blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
        df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
        if datetime.date(int(end_month[:4]), int(end_month[5:]), 1) >= datetime.date(2021, 12, 1):
            try:
                df = df.sample(n=100)
            except:
                print(traceback.format_exc())
        target_df = target_df.append(df)

    

2021/08/fused/pv_v1/BM; TP; YA_251_251.csv
Traceback (most recent call last):
  File "<ipython-input-44-f179479de6c6>", line 39, in <module>
    df = df.sample(n=100)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/generic.py", line 4970, in sample
    locs = rs.choice(axis_length, size=n, replace=replace, p=weights)
  File "mtrand.pyx", line 946, in numpy.random.mtrand.RandomState.choice
ValueError: Cannot take a larger sample than population when 'replace=False'

2021/08/fused/pv_v1/BM_252_256.csv
2021/08/fused/pv_v1/BM_257_261.csv
2021/08/fused/pv_v1/BM_262_266.csv
2021/08/fused/pv_v1/BM_267_271.csv
2021/08/fused/pv_v1/BM_272_275.csv
2021/08/fused/pv_v1/BM_276_278.csv
2021/08/fused/pv_v1/BM_279_283.csv
2021/08/fused/pv_v1/BM_284_287.csv
Traceback (most recent call last):
  File "<ipython-input-44-f179479de6c6>", line 39, in <module>
    df = df.sample(n=100)
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/generic.py", line 4970

In [45]:
target_df

filename        lat         lon    year  month1  month2  \
0   BM; TP; YA_251_251.csv -22.428474  117.695666  2021.0     0.0     0.0   
1   BM; TP; YA_251_251.csv -22.428563  117.695655  2021.0     0.0     0.0   
2   BM; TP; YA_251_251.csv -22.429561  117.695729  2021.0     0.0     0.0   
3   BM; TP; YA_251_251.csv -22.429571  117.695827  2021.0     0.0     0.0   
4   BM; TP; YA_251_251.csv -22.429650  117.695718  2021.0     0.0     0.0   
..                     ...        ...         ...     ...     ...     ...   
44          YA_442_446.csv -22.792758  119.174279  2021.0     0.0     0.0   
0           YA_447_449.csv -22.786564  119.194030  2021.0     0.0     0.0   
1           YA_447_449.csv -22.786573  119.194128  2021.0     0.0     0.0   
2           YA_447_449.csv -22.786653  119.194021  2021.0     0.0     0.0   
3           YA_447_449.csv -22.786662  119.194118  2021.0     0.0     0.0   

    month3  month4  month5  month6  ...  tp_prev2      tp_prev3     swvl1  \
0      0.0     0.0     0.0     0.0  ...  0.000820  1.612650e-03  0.137926   
1      0.0     0.0     0.0     0.0  ...  0.000820  1.612460e-03  0.137808   
2      0.0     0.0     0.0     0.0  ...  0.000820  1.610119e-03  0.136400   
3      0.0     0.0     0.0     0.0  ...  0.000820  1.609993e-03  0.136346   
4      0.0     0.0     0.0     0.0  ...  0.000820  1.609930e-03  0.136282   
..     ...     ...     ...     ...  ...       ...           ...       ...   
44     0.0     0.0     0.0     0.0  ...  0.000001  9.555370e-07  0.067081   
0      0.0     0.0     0.0     0.0  ...  0.000001  9.555370e-07  0.072690   
1      0.0     0.0     0.0     0.0  ...  0.000001  9.555370e-07  0.072727   
2      0.0     0.0     0.0     0.0  ...  0.000001  9.555370e-07  0.072716   
3      0.0     0.0     0.0     0.0  ...  0.000001  9.555370e-07  0.072754   

    swvl1_prev     co2  co2_prev  season_ch_3mon  season_ch_6mon  \
0     0.147378  412.11    413.48       -0.003516       -0.031963   
1     0.147254  412.11    413.48       -0.010382        0.028596   
2     0.145780  412.11    413.48        0.028810       -0.027558   
3     0.145722  412.11    413.48        0.023333        0.050769   
4     0.145657  412.11    413.48        0.016921       -0.028915   
..         ...     ...       ...             ...             ...   
44    0.087686  414.01    414.01        0.139482       -0.043228   
0     0.093049  414.01    414.01        0.155134        0.074821   
1     0.093085  414.01    414.01        0.124345        0.101451   
2     0.093074  414.01    414.01        0.288882        0.111968   
3     0.093110  414.01    414.01        0.329196        0.159130   

    season_ch_9mon  season_ch_12mon  
0         0.394423         0.143006  
1         0.620472         0.099946  
2         0.247526         0.126470  
3         0.111573         0.153991  
4         0.239548         0.163011  
..             ...              ...  
44       -0.094457        -0.087374  
0         0.020352        -0.027388  
1         0.011121        -0.017936  
2        -0.048652        -0.080949  
3        -0.029883        -0.072867  

[101960 rows x 46 columns]

In [34]:
import datetime
from dateutil.relativedelta import relativedelta
from vdsm_util import get_blob_client
import pickle as pkl
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO

end_month = '2021-12'
veg_type = 'pv'

training_path = 'training/traininginput_{}_v2'.format(veg_type)

blob_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_client.get_container_client("data")
blob_names = container_client.list_blobs(name_starts_with=training_path)

end_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1)
start_month_dt = datetime.date(int(end_month[:4]), int(end_month[5:]), 1) + relativedelta(months=-12)

months = []
month_dt = start_month_dt
while month_dt < end_month_dt:
    month_dt = month_dt + relativedelta(months=1)
    months.append(str(month_dt)[:7])

target_df = pd.DataFrame()

for blob_name in blob_names:
    print(blob_name['name'])
    blob_client = get_blob_client(filename=blob_name, conn_str=conn_str)
    df = pd.read_csv(StringIO(blob_client.download_blob().content_as_text()))
    df['year'] = df['year'].astype('int')
    df['month'] = df['month'].astype('int')
    filt = df['year'] == int(months[0][:4]) # & (df['month'] == int(months[0][5:]))
    # df = blob_client.download_blob().read_all().deecode()
    print(df[filt])
    # training_dataset = pkl.loads(blob_client.download_blob().content_as_bytes())


    # months = []
    # month_dt = start_date
    # while month_dt < end_date:
    #     months.append(str(month_dt)[:7])
    #     month_dt = month_dt + relativedelta(months=1)

training/traininginput_pv_v2/BM; TP; YA_251_251.csv
Empty DataFrame
Columns: [year, month, evi, evi_prev, u10, u10_prev, v10, v10_prev, t2m, t2m_prev, lai_lv, lai_lv_prev, src, src_prev, ssr, ssr_prev, sp, sp_prev, e, e_prev, tp, tp_prev, tp_prev2, tp_prev3, swvl1, swvl1_prev, co2, co2_prev, lat_4326, lon_4326, season_ch_3mon, season_ch_6mon, season_ch_9mon, season_ch_12mon, actual_3m, actual_6m, actual_9m, actual_12m]
Index: []

[0 rows x 38 columns]
training/traininginput_pv_v2/BM_252_256.csv
Empty DataFrame
Columns: [year, month, evi, evi_prev, u10, u10_prev, v10, v10_prev, t2m, t2m_prev, lai_lv, lai_lv_prev, src, src_prev, ssr, ssr_prev, sp, sp_prev, e, e_prev, tp, tp_prev, tp_prev2, tp_prev3, swvl1, swvl1_prev, co2, co2_prev, lat_4326, lon_4326, season_ch_3mon, season_ch_6mon, season_ch_9mon, season_ch_12mon, actual_3m, actual_6m, actual_9m, actual_12m]
Index: []

[0 rows x 38 columns]
training/traininginput_pv_v2/BM_257_261.csv
Empty DataFrame
Columns: [year, month, evi, evi_prev

In [ ]:
training_dataset